In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [ ]:
import findspark
findspark.init()
findspark.find()

In [ ]:
import pyspark

## Стратифицированная выборка

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate() 

data_dir = "./data"
data_file_name = "train_data.csv"
data_path = os.path.join(data_dir,data_file_name)

raw_df = spark.read.load(data_path,format="csv",delimiter="|",header=True)

In [ ]:
# sample 
sampled_df = raw_df.stat.sampleBy("label",
                     {"0":0.02,"1":0.02},
                     seed=100)

In [ ]:
sample_file_name = "sampled_data.csv"
ouptput_dir = "./data"
arr = sampled_df.write.format('csv').option('header',True).mode('overwrite').option('sep',',')

In [ ]:
# arr.save(os.path.join(ouptput_dir,sample_file_name))

## Split обучающей выборки

In [ ]:
data_dir = "./data/sampled_data.csv/"

df = pd.DataFrame()
for file in os.listdir(data_dir):    
    if file[-3:] == "csv":
        df = pd.concat((df,pd.read_csv(os.path.join(data_dir,file))),axis=0)

In [ ]:
df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:],df['label'],test_size=0.25,random_state=42,stratify=df['label'])

In [ ]:
train_df = pd.concat((y_train, X_train), axis=1)
test_df = pd.concat((y_test, X_test), axis=1)

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
train_df.to_csv("./data/train.csv",index=False)

In [ ]:
test_df.to_csv("./data/test.csv",index=False)